Importamos librerias

In [1]:
import requests
import os
import base64
import json
import pandas as pd
from fastcore.all import *
from tqdm.notebook import tqdm, trange
import shutil

Me logeo a la API de Mercado Libre

In [2]:
# Busco mis keys
with open('keys.txt') as f:
    key = f.read().strip()
    
# Seteo el site de Mercado Libre a scrapper
site_id = 'MLA'

# Ingreso con mis datos
bearer_token = base64.b64encode(bytes("'Bearer {}'".format(key), encoding='utf-8'))
header = {'Authorization': bearer_token}

Categorias

In [3]:
url = 'https://api.mercadolibre.com/sites/MLA/categories'
r = requests.get(url, headers=header)
cats = r.json()
cats = pd.DataFrame(cats)

In [4]:
# Busco la categoria de Autos
cats[cats.name.str.lower().str.contains('auto')]

,id,name
6,MLA1743,"Autos, Motos y Otros"


Subcategorias

In [5]:
# Busco dentro de las subcategorías las de autos usados
cat_id = cats.loc[cats.name.str.lower().str.contains('auto'), 'id'].values[0]
url = f'https://api.mercadolibre.com/categories/{cat_id}'
r = requests.get(url, headers=header)
scats = pd.DataFrame(r.json()['children_categories']).sort_values('total_items_in_this_category')
scats

,id,name,total_items_in_this_category
4,MLA51547,Colectivos,342
11,MLA93430,Semirremolques,830
7,MLA80579,Motorhomes,1049
0,MLA93412,Autos Chocados y Averiados,1069
10,MLA1784,Planes de Ahorro,1572
5,MLA7312,Maquinaria Agrícola,2269
6,MLA405183,Maquinaria Vial,2459
1,MLA1745,Autos de Colección,2578
3,MLA58254,Camiones,2775
12,MLA1907,Otros Vehículos,6564


In [6]:
# Función para buscar todos los atributos de la subcategoria 'Autos Usados'
def get_subcat_attr(scat):
    url = f'https://api.mercadolibre.com/categories/{scat}/attributes'
    r= requests.get(url, headers=header)
    data = []
    for d in r.json():
        data.append({f'attr_{k}': v for k,v in d.items()})
    for d in data:
        if not 'attr_values' in d:
            d['attr_values'] = [{'id':None, 'name': None}]
        for k in d:
            if k!='attr_values' and k!='attr_tags':
                for av in d['attr_values']:
                    av[k] = d[k]
    data = sum([d['attr_values'] for d in data if 'attr_values' in d], [])
    scat_attr = pd.DataFrame(data).set_index('attr_id')
    return scat_attr

In [7]:
url = f'https://api.mercadolibre.com/categories/MLA1744/attributes'
r= requests.get(url, headers=header)
r.json()[1]

{'id': 'MODEL',
 'name': 'Modelo',
 'tags': {'catalog_required': True, 'required': True},
 'hierarchy': 'PARENT_PK',
 'relevance': 1,
 'value_type': 'string',
 'value_max_length': 255,
 'values': [{'id': '34546', 'name': '1'},
  {'id': '9267', 'name': '10'},
  {'id': '8943', 'name': '100'},
  {'id': '8272', 'name': '1000'},
  {'id': '922967', 'name': '1007'},
  {'id': '59332', 'name': '1020'},
  {'id': '12564', 'name': '1022'},
  {'id': '11530', 'name': '1025'},
  {'id': '11194', 'name': '105'},
  {'id': '41716', 'name': '106'},
  {'id': '2498194', 'name': '1061'},
  {'id': '22472', 'name': '107'},
  {'id': '43829', 'name': '108'},
  {'id': '45327', 'name': '11'},
  {'id': '10320', 'name': '110'},
  {'id': '2468938', 'name': '1102'},
  {'id': '13009', 'name': '111'},
  {'id': '18220', 'name': '112'},
  {'id': '19297', 'name': '115'},
  {'id': '110493', 'name': '121'},
  {'id': '568935', 'name': '124'},
  {'id': '10042', 'name': '125'},
  {'id': '50452', 'name': '126'},
  {'id': '104205

In [8]:
scat = 'MLA1744'
scat_attr = get_subcat_attr(scat)
scat_attr.head()

,id,name,attr_name,attr_hierarchy,attr_relevance,attr_value_type,attr_value_max_length,attr_attribute_group_id,attr_attribute_group_name,attr_hint,metadata,attr_allowed_units,attr_default_unit,attr_tooltip
attr_id,,,,,,,,,,,,,,
BRAND,2253232,AMC,Marca,PARENT_PK,1,string,255.0,FIND,Ficha técnica,Escribe la marca real del producto o 'Genérica...,NaN,NaN,NaN,NaN
BRAND,60244,Acura,Marca,PARENT_PK,1,string,255.0,FIND,Ficha técnica,Escribe la marca real del producto o 'Genérica...,NaN,NaN,NaN,NaN
BRAND,380854,Agrale,Marca,PARENT_PK,1,string,255.0,FIND,Ficha técnica,Escribe la marca real del producto o 'Genérica...,NaN,NaN,NaN,NaN
BRAND,2817419,Aleko,Marca,PARENT_PK,1,string,255.0,FIND,Ficha técnica,Escribe la marca real del producto o 'Genérica...,NaN,NaN,NaN,NaN
BRAND,67695,Alfa Romeo,Marca,PARENT_PK,1,string,255.0,FIND,Ficha técnica,Escribe la marca real del producto o 'Genérica...,NaN,NaN,NaN,NaN


In [9]:
L(scat_attr.index.unique(), use_list = True).filter(lambda x:'VEH' in x)

(#7) ['VEHICLE_YEAR','VEHICLE_TYPE','VEHICLE_BODY_TYPE','IS_INSURED_VEHICLE','HAS_VEHICLE_INSPECTION','VERIFIED_VEHICLES','FIPE_VEHICLE_YEAR']

In [10]:
scat_attr.loc['VEHICLE_YEAR']

id                                    None
name                                  None
attr_name                              Año
attr_hierarchy                   PARENT_PK
attr_relevance                           1
attr_value_type                     number
attr_value_max_length                 18.0
attr_attribute_group_id               FIND
attr_attribute_group_name    Ficha técnica
attr_hint                              NaN
metadata                               NaN
attr_allowed_units                     NaN
attr_default_unit                      NaN
attr_tooltip                           NaN
Name: VEHICLE_YEAR, dtype: object

Search

In [11]:
def search(site_id='MLA', category_id='MLA1744', offset=0, limit=1000, filter=[] ,results=[] , return_filters=False):
    url = f'https://api.mercadolibre.com/sites/{site_id}'
    url = url + f'/search?category={category_id}&offset={offset}'
    for f in filter:
        url += f'&{f["name"]}={f["value"]}'
    r = requests.get(url, headers=header)
    if return_filters:
        return r.json()['available_filters']
    paging = r.json()['paging']

    results += r.json()['results']

    offset += 50
    if offset >= paging['primary_results']:
        return results
    else:
        return search(site_id=site_id, category_id=category_id, offset=offset, limit=limit, filter=filter , results=results, return_filters = False)

In [12]:
def get_year_ranges_for_brand(brand_id):
    filters = search(filter=[{'name': 'BRAND', 'value': brand_id}], results=[], return_filters=True)
    years = L(filters).filter(lambda x:'VEHICLE_YEAR'==x['id'])
    if len(years)==0:
        return[]
    foo = pd.DataFrame(years[0]['values'])
    foo = foo.sort_values('name').reset_index(drop=True)
    ranges = []
    s = 0
    for i,row in foo.iterrows():
        if s==0:
            ranges.append([row['name'], row['name']])
        s += int(row.results)
        if s>1000:
            ranges[-1][1] = row['name']
            s = 0
    ranges[-1][1] = row['name']
    return L(ranges).map('-'.join)


In [13]:
# Check
brand_id = 67695
get_year_ranges_for_brand(brand_id)

(#1) ['1989-2022']

In [14]:
filters = search(filter=[{'name':'BRAND', 'value': brand_id}], results=[], return_filters=True)
years = L(filters).filter(lambda x: 'VEHICLE_YEAR'==x['id'])

In [15]:
foo = scat_attr.reset_index()
foo.loc[foo.id.eq('67695')]

,attr_id,id,name,attr_name,attr_hierarchy,attr_relevance,attr_value_type,attr_value_max_length,attr_attribute_group_id,attr_attribute_group_name,attr_hint,metadata,attr_allowed_units,attr_default_unit,attr_tooltip
4,BRAND,67695,Alfa Romeo,Marca,PARENT_PK,1,string,255.0,FIND,Ficha técnica,Escribe la marca real del producto o 'Genérica...,NaN,NaN,NaN,NaN


In [17]:
results = []
for brand_id in tqdm(scat_attr.loc['BRAND', 'id'].values):
    ranges = get_year_ranges_for_brand(brand_id)
    for years in tqdm(ranges, leave=False):
        results = search(filter=[{'name':'BRAND', 'value': brand_id}, {'name':'VEHICLE_YEAR', 'value': years}], results = results)

  0%|          | 0/198 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [18]:
def results2df(results):
    skip = ['prices','shipping','seller_contact','location','seller','melicoin','original_price','installments',
            'sale_price', 'discounts','melicoin', 'offer_score','offer_share', 'match_score', 'winner_item_id','sellercar_dealer_logo',
            'sellerhome_image_url','accepts_mercadopago','address','category_id', 'domain_id', 'Condición del ítem',
            'adressarea_code','adressphone1','stop_time','permalink','thumbnail','seller_address','attributes']
    df = pd.DataFrame()
    results_pp = []
    for x in tqdm(results):
        temp = {k: v for k,v in x.items() if k not in skip}
        for a in x['attributes']:
            temp[a['name']] = a['value_name']
        for k,v in x['address'].items():
            temp[f'adress{k}'] = v 
        results_pp.append(temp)

    df = pd.DataFrame(results_pp)
    return df

In [19]:
skip = ['prices','shipping','seller_contact','location','seller','melicoin','original_price','installments',
            'sale_price', 'discounts','melicoin', 'offer_score','offer_share', 'match_score', 'winner_item_id','sellercar_dealer_logo',
            'sellerhome_image_url','Combustible')
]

for k,v in results[1]['address'].items():
            {k: v for k,v in results[1]['address'].items() if k not in skip}[f'address{k}'] = v 

In [20]:
results[1]['address'].items()

dict_items([('state_id', 'TUxBUFpPTmFpbnRl'), ('state_name', 'Buenos Aires Interior'), ('city_id', 'TUxBQ0xJTjU4Njg'), ('city_name', 'Lincoln'), ('area_code', ''), ('phone1', '')])

In [21]:
df = results2df(results)
df.head()

  0%|          | 0/72890 [00:00<?, ?it/s]

,id,site_id,title,price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,...,Largo,Capacidad de personas,Potencia,Cilindros,Línea,Horas de uso,Marca del motor,Capacidad de carga,Peso,Combustible
0,MLA1133956632,MLA,Acura Legend V6 3.2l At.- Full Titular,1100000.0,ARS,1,0,classified,free,used,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MLA932476572,MLA,Agrale Marruá Am150 Cd,4000000.0,ARS,1,0,classified,silver,used,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MLA1121119730,MLA,Alfa Romeo Giullieta 1.4 T Distinctive At 2016...,3700000.0,ARS,1,0,classified,gold_premium,used,...,4351 mm,5,170 hp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MLA1127687379,MLA,Alfa Romeo Mito 1.4 Tbi Quadrifoglio Verde 2012,2940000.0,ARS,1,0,classified,gold_premium,used,...,4065 mm,5,170 hp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MLA1136612801,MLA,Alfa Romeo Mito Progression !! Tm,2880000.0,ARS,1,0,classified,gold_premium,used,...,4065 mm,5,78 hp,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#df = pd.DataFrame(results)
#df.to_excel('resultados.xlsx', index=False)

In [22]:
for col in df.columns:
    print(col)

id
site_id
title
price
currency_id
available_quantity
sold_quantity
buying_mode
listing_type_id
condition
thumbnail_id
official_store_id
catalog_product_id
tags
order_backend
use_thumbnail_id
Condición del ítem
Único dueño
Control de tracción
Aire acondicionado
Marca
Puertas
Motor
Tipo de combustible
Kilómetros
Modelo
Transmisión
Versión
Año
adressstate_id
adressstate_name
adresscity_id
adresscity_name
adressarea_code
adressphone1
Cilindrada
Largo
Capacidad de personas
Potencia
Cilindros
Línea
Horas de uso
Marca del motor
Capacidad de carga
Peso
Combustible


In [23]:
df.shape

(72890, 46)

In [24]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72890 entries, 0 to 72889
Data columns (total 46 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     72890 non-null  object 
 1   site_id                72890 non-null  object 
 2   title                  72890 non-null  object 
 3   price                  72890 non-null  float64
 4   currency_id            72890 non-null  object 
 5   available_quantity     72890 non-null  int64  
 6   sold_quantity          72890 non-null  int64  
 7   buying_mode            72890 non-null  object 
 8   listing_type_id        72890 non-null  object 
 9   condition              72890 non-null  object 
 10  thumbnail_id           72890 non-null  object 
 11  official_store_id      7389 non-null   float64
 12  catalog_product_id     62859 non-null  object 
 13  tags                   72890 non-null  object 
 14  order_backend          72890 non-null  int64  
 15  us

In [25]:
df.isna().sum().sort_values()

id                           0
Modelo                       0
Kilómetros                   0
Tipo de combustible          0
adressstate_id               0
Puertas                      0
Marca                        0
adressstate_name             0
adresscity_id                0
adresscity_name              0
Condición del ítem           0
use_thumbnail_id             0
tags                         0
order_backend                0
adressphone1                 0
thumbnail_id                 0
condition                    0
listing_type_id              0
buying_mode                  0
sold_quantity                0
available_quantity           0
currency_id                  0
price                        0
title                        0
site_id                      0
adressarea_code              0
Año                          0
Versión                      1
Transmisión               4584
Motor                     5269
Control de tracción       7154
catalog_product_id       10031
Capacida

In [26]:
df.to_excel('data_api.xlsx', index=False)